In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# A.Introdução

Este projeto foi motivado pela curiosidade de compreender o impacto de variáveis sociais, como o estado onde a pessoa habita, tipo de instituição escolar, situação de conclusão do ensino médio, entre outros, na nota na prova de Matemáticas e suas Tecnologias. Assim, buscamos entender mais precisamente como esses fatores influenciam no resultado do aluno.  A base de dados usada foi a de Microdados do ENEM 2019, do INEP. A base tinha mais 5 milhões de linhas e 136 colunas, cada linha representando um candidato e cada coluna representando uma das variáveis sociais. Portanto, primeiro foi feita uma coleta de uma amostra aleatória e depois uma limpeza dessa amostra. Foram eliminadas as linhas das pessoas que tinham faltado a prova de Matemática, já que esses casos não contribuiam para chegar no objetivo do estudo. Além disso, as colunas das variáveis da base de dados originais foram reduzidas a 11, sendo essas as consideradas mais impactantes para a nota de matemática do aluno no ENEM.

Bibliotecas importadas:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns
import statsmodels.api as sm
from scipy import stats


from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_graphviz
from sklearn.metrics import r2_score


Coleta de uma amostra aleatória do dataset: 
Foi necessário coletar uma porcentagem da base de dados pois ela era muito extensa (mais de 5 milhões de linhas). Além disso, essa coleta precisava ser aleatória pois os dados estavam ordenados por estado.

In [ ]:
# baseado no código elaborado por Davi Reis Vieira de Souza

fonte = '../input/enem-2019/DADOS/MICRODADOS_ENEM_2019.csv' #arquivo

seed = 31123 # define o padrao de aleatoriedade
np.random.seed(seed)

total_linhas = sum(1 for l in open(fonte,encoding="ISO-8859-1") )#conta quantas linhas tem o data set
print(f'Total de Linhas: {total_linhas}')

p = 0.025 
tamanho = int(total_linhas*p)# calcula quanto é 2.5% dos dados 
print(f'Total de Linhas para Estudo: {tamanho}')

index = random.sample(range(1, total_linhas), total_linhas - tamanho)# lista de numeros aleatorios

dados = pd.read_csv(fonte,encoding="ISO-8859-1",delimiter=";",skiprows=index)# selecions linhas da base de dados com base na lista de numero aleatórios
dados.shape

In [ ]:
dados.head(15) # visualizacao da amostra de dados aleatória 

In [ ]:
dados.SG_UF_RESIDENCIA.value_counts() #confirma a presenca de todos os estados 

# B. Minerando Dados e Características do Dataset 

In [ ]:
# como a variável target é a nota de matemática, vamos usar somente os dados dos candidatos que estavam presentes nessa prova (1 indica presença)
dados = dados.loc[dados['TP_PRESENCA_MT']==1]
print(len(dados)) # agora a base de dados reduziu um pouco

In [ ]:
#só as colunas das variáveis features que vamos usar
colunas = ['SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'NU_NOTA_MT']
dados = dados[colunas]
dados.head(10)

In [ ]:
#legenda das variaveis features
variaveis=np.array([["SG_UF_RESIDENCIA", "Sigla da Unidade da Federação de residência"],["NU_IDADE","Idade"], ["TP_SEXO", "Sexo"], ["TP_ESTADO_CIVIL", "Estado Civil"], ["TP_COR_RACA", "Cor/raça"], ["TP_NACIONALIDADE", "Nacionalidade"], ["TP_ST_CONCLUSAO", "Situação de conclusão do Ensino Médio"], ["TP_ANO_CONCLUIU", "Ano de Conclusão do Ensino Médio"], ["TP_ESCOLA", "Tipo de escola do Ensino Médio"], ["TP_ENSINO", "Tipo de instituição que concluiu ou concluirá o Ensino Médio"], ["IN_TREINEIRO", "Indica se o inscrito fez a prova com intuito de apenas treinar seus conhecimentos"]])

variaveis_df=pd.DataFrame(variaveis, columns=["Nome no Data Frame", "Descrição"])

variaveis_df.index = variaveis_df.index + 1
variaveis_df

In [ ]:
#legenda dos valores das variáveis
estadocivil=np.array([[0,"Não informado"],[1, "Solteiro(a)"], [2,"Casado(a)/Mora com companheiro(a)"], [3,"Divorciado(a)/Desquitado(a)/Separado(a)"], [4, "Viúvo(a)"]])

estadocivil_df=pd.DataFrame(estadocivil, columns=["Valor", "Estado Civil"])

display(estadocivil_df.style.hide_index())



cor_raca=np.array([[0,"Não declarado"],[1, "Branca"], [2,"Preta"], [3,"Parda"], [4, "Amarela"], [5, "Indígena"]])

cor_raca_df=pd.DataFrame(cor_raca, columns=["Valor", "Cor/Raça"])

display(cor_raca_df.style.hide_index())


nacionalidade=np.array([[0,"Não informado"],[1, "Brasileiro(a)"], [2,"Brasileiro(a) Naturalizado(a)"], [3,"Estrangeiro(a)"], [4, "Brasileiro(a) Nato(a), nascido(a) no exterior"]])

nacionalidade_df=pd.DataFrame(nacionalidade, columns=["Valor", "Nacionalidade"])

display(nacionalidade_df.style.hide_index())



stconclusao=np.array([[1,"Já concluí o Ensino Médio"],[2, "Estou cursando e concluirei o Ensino Médio em 2019"], [3,"Estou cursando e concluirei o Ensino Médio após 2019"], [4,"Não concluí e não estou cursando o Ensino Médio"]])

stconclusao_df=pd.DataFrame(stconclusao, columns=["Valor", "Situação de Conclusão do Ensino Médio"])

display(stconclusao_df.style.hide_index())



anoconclusao=np.array([[0,"Não informado"],[1, "2018"], [2,"2017"], [3,"2016"], [4, "2015"], [5, "2014"],[6, "2013"],[7, "2012"],[8, "2011"],[9, "2010"],[10, "2009"],[11, "2008"],[12, "2007"],[13, "Antes de 2007"]])

anoconclusao_df=pd.DataFrame(anoconclusao, columns=["Valor", "Ano de Conclusão do Ensino Médio"])

display(anoconclusao_df.style.hide_index())

        
tipoescola=np.array([[1, "Não Respondeu"], [2,"Pública"], [3,"Privada"]])

tipoescola_df=pd.DataFrame(tipoescola, columns=["Valor", "Tipo de Escola"])

display(tipoescola_df.style.hide_index())

        
tipoensino=np.array([[1, "Ensino Regular"], [2,"Educação Especial - Modalidade Substitutiva"]])

tipoensino_df=pd.DataFrame(tipoensino, columns=["Valor", "Tipo de Ensino"])

display(tipoensino_df.style.hide_index())


treineiro=np.array([[0, "Não"], [1,"Sim"]])

treineiro_df=pd.DataFrame(treineiro, columns=["Valor", "Se é treineiro"])

display(treineiro_df.style.hide_index())

# Análise exploratória: cruzando cada variável feature com a variável target (Nota de matemática)

Observação: Cada vez que o código é rodado, uma nova amostra aleatória de dados é gerada, portanto a análise a seguir pode ter pequenas impertinências dependendo da amostra gerada.

In [ ]:
#1) Sigla da Unidade da Federação de residência
dados.groupby("SG_UF_RESIDENCIA").NU_NOTA_MT.describe()


Conclusões:
1. O número de candidatos por estado varia bastante, o que pode estar relacionado com o número de habitantes daquele estado, seu nível de desenvolvimento ou a amostra coletada.
2.  É possível prever que a média dos estados que tem 0 como valor mínimo será prejudicada em relação aos estados que tem por volta de 360 como mínimo. Os valores 0 indicam que alguma prova foi anulada (por cola por exemplo), pois pelo modelo TRI, é impossível tirar 0, mesmo deixando todas as questoes em branco ou errando todas.


In [ ]:
#2)Idade
faixasdeidade=pd.cut(dados.NU_IDADE, bins=range(0,100,10))
dados["faixas_de_idade"] = faixasdeidade


In [ ]:
dados.groupby("faixas_de_idade").NU_NOTA_MT.describe()

Conclusões:
1. É estranho o fato de existirem algumas pessoas com idades entre 0 e 10 anos.
2. Como esperado, a maior parte da amostra tem de 10 a 20 anos, pois nessa faixa etária acontece a conclusão do ensino médio e, para uma parte da população, a entrada na faculdade. E também como esperado essa faixa etária tem a maior média de nota na prova de matemática, já que normalmente o contato com os conteúdos da prova foi mais recente do que para outras faixas etárias.
    

In [ ]:
#3)Sexo
dados.groupby("TP_SEXO").NU_NOTA_MT.describe()

Conclusões:

1. Existem mais mulheres do que homens prestando a prova.
2. A média das notas das mulheres é menor que a dos homens.

In [ ]:
dados_m = dados.loc[dados['TP_SEXO']=='M']
dados_f = dados.loc[dados['TP_SEXO']=='F']

plt.figure(figsize=(17,5))

plt.subplot(131)
plt.hist(dados.NU_NOTA_MT, density = True)
plt.title('Notas de matematica')
plt.ylabel('densidade')
plt.xlabel("nota")

plt.subplot(132)
plt.hist(dados_m.NU_NOTA_MT, density = True)
plt.title('Notas de matematica dos homens')
plt.ylabel('densidade')
plt.xlabel("nota")

plt.subplot(133)
plt.hist(dados_f.NU_NOTA_MT, density = True)
plt.title('Notas de matematica das mulheres')
plt.ylabel('densidade')
plt.xlabel("nota")
plt.show()

In [ ]:
#4) Estado Civil
display(estadocivil_df.style.hide_index())#note a legenda
dados.groupby("TP_ESTADO_CIVIL").NU_NOTA_MT.describe()


Conclusões

1. A grande maioria dos candidatos é solteiro(a),o que faz sentindo, pois são jovens que estão concluindo o ensino médio.
2. É curioso o fato de que as pessoas que são Divorciadas ou Viúvas (3 e 4) tem a nota mínima de 360, ou seja ninguém desses grupos cometeu alguma infração na prova e teve a prova zerada.

In [ ]:
#5) Cor/Raça

display(cor_raca_df.style.hide_index())

dados.groupby("TP_COR_RACA").NU_NOTA_MT.describe()


Conclusões:

1.  A maior parte dos prestadores da prova são pardos, em torno de 42mil pessoas, sendo a segunda maior parcela de pessoas brancas. Apenas 11mil pessoas pretas fizeram a prova, ou seja, aproximadamente 12% do total
2. A  maior média de notas na prova de matemática é do grupo de pessoas brancas, o que comprova a estrutura racista brasileira em que a população branco tem maior acesso à um estudo de qualidade.

In [ ]:
#6) Nacionalidade
display(nacionalidade_df.style.hide_index())

dados.groupby("TP_NACIONALIDADE").NU_NOTA_MT.describe()

Conclusões:

1. A maior parte dos prestadores da prova são brasileiros, e este grupo também é o único com integrantes que cometeram infrações e tiveram a prova zerada.
2. É curioso o fato de que os Brasileiros natos nascidos no exterior (pessoas que são filhos de brasileiros porém nasceram no exterior) tem a maior média das notas das provas de matemática.

In [ ]:
#8) Situação de conclusão de ensino médio
display(stconclusao_df.style.hide_index())

dados.groupby("TP_ST_CONCLUSAO").NU_NOTA_MT.describe()

Conclusões:

1. Como o esperado, a menor média de notas pertence a pessoas que abandonaram o ensino no meio de suas vidas escolares, pois não foram expostas a todos os conteúdos. 
2. A maior parte dos prestadores da prova já conluiram o ensino médio.

In [ ]:
#9) Ano de conclusao do ensino médio
display(anoconclusao_df.style.hide_index())

dados.groupby("TP_ANO_CONCLUIU").NU_NOTA_MT.describe()


Conclusões:
1. É curioso o fato de que a maior parte dos candidatos tem o seu ano de conclusão do Ensino Médio não informado
2. As médias por ano de conclusão do Ensino Médio estão bem próximas uma da outra.

In [ ]:
#10) Tipo de escola no ensino médio
display(tipoescola_df.style.hide_index())

dados.groupby("TP_ESCOLA").NU_NOTA_MT.describe()


Conclusões:
1. A maior parte dos candidatos não respondeu de qual tipo de escola eles fazem parte.
2. A média dos alunos de escolas privadas é consideravelmente maior, o que é um indicativo de qualidade comparativo entre escolas públicas e privadas no Brasil.

In [ ]:
publica = dados.loc[dados['TP_ESCOLA']==2]
privada = dados.loc[dados['TP_ESCOLA']==3]

plt.figure(figsize=(17,5))


plt.subplot(132)
plt.hist(publica.NU_NOTA_MT,bins=range(0,1000, 50), density = True)
plt.title('Notas das pessoas de escola pública')
plt.ylabel('densidade')
plt.xlabel("nota")

plt.subplot(133)
plt.hist(privada.NU_NOTA_MT, bins=range(0,1000, 50),density = True)
plt.title('Notas das pessoas de escola privada')
plt.ylabel('densidade')
plt.xlabel("nota")
plt.show()

In [ ]:
#11) Tipo de ensino no ensino médio
display(tipoensino_df.style.hide_index())

dados.groupby("TP_ENSINO").NU_NOTA_MT.describe()



Conclusões:

1. A maior  parte dos alunos são do Ensino Regular. E a média desses alunos é maior do que os alunos de educação especial.

In [ ]:
#12) Se é ou não treinero
display(treineiro_df.style.hide_index())

dados.groupby("IN_TREINEIRO").NU_NOTA_MT.describe()


Conclusões:

1. A maior parte dos alunos que prestam a prova não são treineiros. 
2. Os treineiros possume maior média, talvez por estarem menos nervosos na prova

In [ ]:
treineiro_sim = dados.loc[dados['IN_TREINEIRO']==1]
treineiro_nao = dados.loc[dados['IN_TREINEIRO']==0]

plt.figure(figsize=(17,5))


plt.subplot(132)
plt.hist(treineiro_sim.NU_NOTA_MT, density = True)
plt.title('Notas dos treineiros')
plt.ylabel('densidade')
plt.xlabel("nota")

plt.subplot(133)
plt.hist(treineiro_nao.NU_NOTA_MT, density = True)
plt.title('Notas dos não treineiros')
plt.ylabel('densidade')
plt.xlabel("nota")
plt.show()

 **Storytelling**

In [ ]:
sns.pairplot(dados, height=1.5);#fazendo o cruzamento de todas as variáveis entre si

De maneira geral, os gráficos são bem dispersos, indicando que existe fraca covariância entre as variáveis. O mesmo se nota na última coluna, que mostra o cruzamento de cada variável com a variável target. No canto inferior direito, há um histograma das notas de Matemática, que pode ser melhor visualizado a seguir.

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(dados.NU_NOTA_MT, bins=50,density = True);
plt.title("Histograma das notas de Matemática")

Aqui, pode-se perceber como as notas começam a partir de 300-350 aproximadamente, reforçando que esse é foi valor mínimo pelo modelo TRI. Os zeros presentes são provenientes de provas anuladas. Nota-se também que a maior densidade está nos valores próximos a 500 e que partir do pico, existe um decaimento da densidade num formato de cauda parecido com uma normal.

 # C. Modelos de Predição e Estatísticas de Validação 

In [ ]:
#tratando os dados para o uso das tecnicas de predicao.Todos valores devem ser numéricos para a aplicação dessas técnicas
k=0
dados['Masculino'] = 0
dados['Feminino'] = 0
for i in dados['TP_SEXO']:
    if i == 'M':
        dados.iloc[k,13] = 1
    else:
        dados.iloc[k,14] = 1
    k += 1
dados = dados.drop(columns=['TP_SEXO'])

In [ ]:
dados
# observar ultimas colunas: Agora as strings M e F foram transformadas em valores de 1 e 0 (verdadeiro e falso)

In [ ]:
#fazendo o mesmo com a variável estado
Estados = {'AC':0,'AL':1,'AM':2,'AP':3,'BA':4,'CE':5,'DF':6,'ES':7,'GO':8,'MA':9,'MT':10,'MS':11,'MG':12,'PA':13,'PB':14,'PR':15,'PE':16,'PI':17,'RR':18,'RO':19,'RJ':20,'RN':21,'RS':22,'SC':23,'SP':24,'SE':25,'TO':26}
for i in Estados:
    dados[i]=0

In [ ]:
t = 0
for i in dados['SG_UF_RESIDENCIA']:
    for j,k in Estados.items():
        if i == j:
            dados.iloc[t,14+k]=1
            t+=1
dados = dados.drop(columns=['SG_UF_RESIDENCIA'])

In [ ]:
dados = dados.dropna()#tirando NaNs

In [ ]:
dados

In [ ]:
train_size = int(len(dados)*0.8)#define treinamento como 80% dos dados
Features = dados[['NU_IDADE', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO','Masculino','Feminino','AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MT','MS','MG','PA','PB','PR','PE','PI','RR','RO','RJ','RN','RS','SC','SP','SE','TO']]
Target = dados['NU_NOTA_MT']

In [ ]:
X_train, y_train, X_test, y_test = Features[:train_size], Target[:train_size], Features[train_size:], Target[train_size:]#cria subdatasets de treinamento e teste para features e target


Modelo de Predição 1: A técnica de Regressão Linear tem como objetivo determinar os coeficientes da função polinomial que recebe as features (informações do candidato) como input e calcula o target (nota de matematica) no output. Para a validação desse modelo, precisamos validar suas suposições, como a de que os erros seguem uma distribuição normal. Para isso, são utilizados 2 métodos, sendo estes, teste Omnibus e teste Jarque-Bera. Também é feito o teste "t": valor P, no qual as features não relevantes são descartadas. Para esse teste, o alpha definido foi de 5%. Por fim, para analisar o desempenho da regressão podemos utilizar o valor R-squared eum gráfico que relaciona os dados de teste com os dados preditos. 

In [ ]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

results = regress(X_train, y_train)
results.summary()

****Atenção: como a amostra de dados gerada é aleatória, os valores-p mudam para cada amostra. Assim, a análise a seguir foi feita com base numa amostra específica gerada****

Teste valor P : Com base no teste do valores p (observados na coluna P>|t|), em que se deve verificar se o valor-p é menor que alpha(5%), retira-se as variáveis TP_ESTADO_CIVIL, TP_ENSINO, IN_TREINEIRO, Sexo e todos os estados. Assim, resta somente as variáveis que aparentam causar maior impacto na anota de Matemática.

In [ ]:
Features = dados[['NU_IDADE', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA']]

In [ ]:
results = regress(X_train, y_train)
results.summary()

Agora, pode-se realizar os testes Omnibus e Jarque-Bera. Ambos os testes consistem em analisar se a Prob(Omnibus) ou a Jarque-Bera (JB) são maiores que alpha(5%). Como esse não é o caso, podemos concluir que os resíduos não seguem uma distribuição normal, quebrando uma das suposições do modelo de regressão linear. 

Mesmo com essa suposição quebrada, uma maneira de compreender o desempenho do modelo é com R-squared. Na tabela com o resumo dos resultados, temos que esse parâmetro é 0.209, o que significa uma baiza adequação ao cenário estudado, visto que varia de 0 a 1, sendo 1 o desejado.

Finalmente, como uma das suposições do modelo não é verdadeira e R-squares deixa a desejar, o modelo não pode ser considerado válido, ou seja, não podemos usar o polinômio com coeficientes encontrados para prever a nota de matemática. O modelo linear não é ideal para esse contexto. Uma possível solução seria o uso de um modelo de regressão mais adequado e robusto. 

Modelo de Predição 2:
A técnica Random Forest Regression é ideal para casos de datasets com muitas features, pois o modelo consiste na criação de "Árvores" que separam, para cada variável, os dados em subgrupos (galhos). Inicialmente, o método escolhe, a partir de uma amostra aleatória, a feature que gera maior impacto na nota de matematica (target), ou seja, a mudança dessa feature influencia diretamente no valor da nota. Essa feature será o fator para a criação de subgrupos. Estes subgrupos passarão pelo mesmo processo de separação, agora com a próxima feature mais relevante, até que todas as features tenham sido aplicadas na árvore. Esse processo é repetido até que se use todos os dados de treinamento, formando algumas árvores. Assim, os dados de teste de uma pessoa são designados a todas as árvores para prever sua nota de matemática. A nota prevista será a média de todos os valores gerado pelas árvores.  

In [ ]:
regr = RandomForestRegressor(max_depth=40, random_state=42) #define hiperparametros


In [ ]:
regr.fit(X_train, y_train) # treina o modelo


In [ ]:
y_predict = regr.predict(X_test)# testa o modelo
print(y_predict)# notas previstas

In [ ]:
plt.scatter(y_test, y_predict)
plt.title("Y teste vs Y predito")
plt.ylabel("Y predito")
plt.xlabel("Y teste")
plt.show()

Num cenário ideal, esse gráfico deveria formar uma linha crescente, ou seja, os valores do teste(em x) seriam os mesmos do predito(em y). É possível perceber que o gráfico acima está bem longe desse formato ideal, mas mesmo assim, demonstra algum comportamento nesse sentido. 

In [ ]:
regr.score(X_test, y_test)

O valor acima é o R-squared do modelo Random Forest. Novamente, entende-se que está bem distante do ideal, que seria 1.

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(121)
plt.hist(y_predict, bins=range(0,1000,50),density = True);

plt.title("Histograma das notas previstas de Matemática")
plt.ylabel("densidade")


plt.subplot(122)

plt.hist(y_test, bins=range(0,1000,50),density = True);

plt.title("Histograma das notas reais de Matemática")
plt.ylabel("densidade")


Apesar de não ser uma boa estratégia para avaliar teoricamente as predições do modelo,a visualização do histograma ofere uma perspectiva mais tangível.

# E. Conclusão 

Conclusao:
    Ambos os modelos se mostraram incapazes de prever a nota de Matemática do Enem de 2019. O primeiro, a Regressão Linear, é um modelo bem simples e já era esperado que não fosse gerar bons resultados, visto que busca encontrar uma única funcão capaz de representar os resultados, que variam de acordo com 11 variáveis diferentes. Já com segundo modelo, o Random Forest Regression, esperava-se melhores resultados, pois funciona de maneira mais complexa e aleatória, gerando múltiplos resultados para cada candidato e depois avaliando-os. Mesmo assim, não obtivemos sucesso, já que os dados previstos variaram significativamente em relação aos testados. Uma possível saída poderia ter sido escolher outras variáveis como features, como por exemplo a notas nas outras provas. É importante destacar que a escolha das features foi feita com base nas nossa suposições do grau de relação entre certa condição social e o desempenho do aluno, e isso pode ser sido um equívoco inicial.

# F. Referências Bibliográficas 

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html#sphx-glr-auto-examples-model-selection-plot-cv-predict-py
https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html
https://towardsdatascience.com/a-quick-and-dirty-guide-to-random-forest-regression-52ca0af157f8#:~:text=Random%20forest%20is%20a%20type,prediction%20of%20the%20individual%20trees.
https://medium.com/swlh/random-forest-and-its-implementation-71824ced454f
https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem



Trabalho em grupo:
Trabalhando no kaggle, tivemos problemas em gerar commits de pessoas diferentes e optamos por trabalhar em uma conta só. Assim, elaboramos uma pequena apresentação das subdivisões de trabalho. 

Ana:
*  Implementou o código que permite coletar uma amostra aleatória da base de dados com a ajuda do Davi Reis. 
* Gerou os gráficos/tabelas do cruzamento de cada feature com a target.
* Legenda das variaveis e dos valores das variaveis.
* Storytelling
* Analise critica


Luana:
* Comentou as tabelas cruzadas, explorando a relação que cada feature poderia ter com a variável target. 
* Escreveu a introdução.
* Referencias bibliograficas
* Titulos e legendas dos graficos

Pietro: 
* Implementou o modelo random forest
* Implementou o modelo de regressao linear
* Texto de descricao dos dois modelos
* Preparou o dataframe para os modelos de predicao
* Analise critica 
